In [1]:
import pandas as pd
import sqlite3

<b> Read Data File of Old File </b>

In [2]:
df_old = pd.read_csv("datasets/Owned Club Community Benefits Review 09212020 Preprocessed.csv",dtype=str)

In [3]:
df_old = df_old[['Community','Club','Benefit Tier','Golf Benefit','Availability','Advance','Dining Benefit','Dining Availability']]


In [4]:
%%time
conn = sqlite3.connect("TempOwnClubFiles.db")
cur = conn.cursor()
df_old.to_sql("OwnClub_Data", conn, if_exists="replace")

Wall time: 273 ms


C:\Users\admin\anaconda3\lib\site-packages\pandas\core\generic.py:2653: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [6]:
query = """select *
		  from OwnClub_Data 
		  where trim(upper(OwnClub_Data.'Benefit Tier')) like '%PRESTONWOOD%'
		  and trim(upper(OwnClub_Data.'Golf Benefit')) in ('NO ACCESS','N/A','')
		  and (trim(upper(OwnClub_Data.Availability)) not in ('NO ACCESS','N/A','')
		  or trim(upper(OwnClub_Data.Advance)) not in ('NO ACCESS','N/A',''))"""

rs_benefit = pd.read_sql_query(query, conn)
rs_benefit.to_csv("outputs/delta.csv",index=False)

In [ ]:
query = """select 
		  trim(upper(OwnClub_New.Community)) ,
		  trim(upper(OwnClub_New.Club)) , 
		  trim(upper(OwnClub_New.'Benefit Tier')) ,count(*) 
		  from OwnClub_New 
		  group by trim(upper(OwnClub_New.Community)) ,
		  trim(upper(OwnClub_New.Club)) , 
		  trim(upper(OwnClub_New.'Benefit Tier')),
		  trim(upper(OwnClub_New.'Golf Benefit')),
		  trim(upper(OwnClub_New.Availability)),
		  trim(upper(OwnClub_New.Advance)),
		  trim(upper(OwnClub_New.'Dining Benefit')),
		  trim(upper(OwnClub_New.'Dining Availability'))
		  having count(*) > 1"""

rs_benefit = pd.read_sql_query(query, conn)
rs_benefit.to_csv("outputs/new_duplicates.csv",index=False)

In [ ]:
query = """select 
		  trim(upper(OwnClub_Old.Community)) ,
		  trim(upper(OwnClub_Old.Club)) , 
		  trim(upper(OwnClub_Old.'Benefit Tier')) ,count(*) 
		  from OwnClub_Old 
		  group by trim(upper(OwnClub_Old.Community)) ,
		  trim(upper(OwnClub_Old.Club)) , 
		  trim(upper(OwnClub_Old.'Benefit Tier')),
		  trim(upper(OwnClub_Old.'Golf Benefit')),
		  trim(upper(OwnClub_Old.Availability)),
		  trim(upper(OwnClub_Old.Advance)),
		  trim(upper(OwnClub_Old.'Dining Benefit')),
		  trim(upper(OwnClub_Old.'Dining Availability'))
		  having count(*) > 1"""

rs_benefit = pd.read_sql_query(query, conn)
rs_benefit.to_csv("outputs/old_duplicates.csv",index=False)

In [ ]:
query = """with op as (select trim(upper(OwnClub_Old.Community)) as Community,
		 trim(upper(OwnClub_Old.Club)) as Club,
		 trim(upper(OwnClub_Old.'Benefit Tier')) as BenefitTier,
		 trim(upper(OwnClub_Old.'Golf Benefit')) as GolfBenefit,
		 trim(upper(OwnClub_Old.Availability)) as Availability,
		 trim(upper(OwnClub_Old.Advance)) as Advance,
		 trim(upper(OwnClub_Old.'Dining Benefit')) as DiningBenefit,
		 trim(upper(OwnClub_Old.'Dining Availability')) as DiningAvailability
		 from OwnClub_Old
		 except
		 select trim(upper(OwnClub_New.Community)) as Community,
		 trim(upper(OwnClub_New.Club)) as Club,
		 trim(upper(OwnClub_New.'Benefit Tier')) as BenefitTier,
		 trim(upper(OwnClub_New.'Golf Benefit')) as GolfBenefit,
		 trim(upper(OwnClub_New.Availability)) as Availability,
		 trim(upper(OwnClub_New.Advance)) as Advance,
		 trim(upper(OwnClub_New.'Dining Benefit')) as DiningBenefit,
		 trim(upper(OwnClub_New.'Dining Availability')) as DiningAvailability
		 from OwnClub_New) 
		 select op.Community,op.Club,op.BenefitTier,
		 GolfBenefit ,
		 op.Availability ,
		 op.Advance ,
		 DiningBenefit ,
		 DiningAvailability 
		 from op left join OwnClub_new as nw
		 on (op.Community=trim(upper(nw.Community))
		 and op.Club=trim(upper(nw.Club))
		 and op.BenefitTier=trim(upper(nw.'Benefit Tier')))
		 where nw.Community is null and nw.Club is null and nw.'Benefit Tier' is null"""

rs_benefit = pd.read_sql_query(query, conn)
rs_benefit.to_csv("outputs/diff_missing.csv",index=False)

In [ ]:
query = """with op as (select trim(upper(OwnClub_Old.Community)) as Community,
		 trim(upper(OwnClub_Old.Club)) as Club,
		 trim(upper(OwnClub_Old.'Benefit Tier')) as BenefitTier,
		 trim(upper(OwnClub_Old.'Golf Benefit')) as GolfBenefit,
		 trim(upper(OwnClub_Old.Availability)) as Availability,
		 trim(upper(OwnClub_Old.Advance)) as Advance,
		 trim(upper(OwnClub_Old.'Dining Benefit')) as DiningBenefit,
		 trim(upper(OwnClub_Old.'Dining Availability')) as DiningAvailability
		 from OwnClub_Old
		 except
		 select trim(upper(OwnClub_New.Community)) as Community,
		 trim(upper(OwnClub_New.Club)) as Club,
		 trim(upper(OwnClub_New.'Benefit Tier')) as BenefitTier,
		 trim(upper(OwnClub_New.'Golf Benefit')) as GolfBenefit,
		 trim(upper(OwnClub_New.Availability)) as Availability,
		 trim(upper(OwnClub_New.Advance)) as Advance,
		 trim(upper(OwnClub_New.'Dining Benefit')) as DiningBenefit,
		 trim(upper(OwnClub_New.'Dining Availability')) as DiningAvailability
		 from OwnClub_New) 
		 select op.Community,op.Club,op.BenefitTier,
		 trim(upper(nw.'Golf Benefit')) as GolfBenefit_new,GolfBenefit as GolfBenefit_old,
		 trim(upper(nw.Availability)) as Availability_new,op.Availability as Availability_old,
		 trim(upper(nw.Advance)) as Advance_new,op.Advance as Advance_old,
		 trim(upper(nw.'Dining Benefit')) as DiningBenefit_new,DiningBenefit as DiningBenefit_old,
		 trim(upper(nw.'Dining Availability')) as DiningAvailability_new,DiningAvailability as DiningAvailability_old
		 from OwnClub_new as nw,op
		 where op.Community=trim(upper(nw.Community))
		 and op.Club=trim(upper(nw.Club))
		 and op.BenefitTier=trim(upper(nw.'Benefit Tier'))"""

rs_benefit = pd.read_sql_query(query, conn)
rs_benefit.to_csv("outputs/diff_exists.csv",index=False)

In [ ]:
query = """with op as (select trim(upper(OwnClub_New.Community)) as Community,
trim(upper(OwnClub_New.Club)) as Club,
trim(upper(OwnClub_New.'Benefit Tier')) as BenefitTier,
trim(upper(OwnClub_New.'Golf Benefit')) as GolfBenefit,
trim(upper(OwnClub_New.Availability)) as Availability,
trim(upper(OwnClub_New.Advance)) as Advance,
trim(upper(OwnClub_New.'Dining Benefit')) as DiningBenefit,
trim(upper(OwnClub_New.'Dining Availability')) as DiningAvailability
from OwnClub_New
except
select trim(upper(OwnClub_Old.Community)) as Community,
trim(upper(OwnClub_Old.Club)) as Club,
trim(upper(OwnClub_Old.'Benefit Tier')) as BenefitTier,
trim(upper(OwnClub_Old.'Golf Benefit')) as GolfBenefit,
trim(upper(OwnClub_Old.Availability)) as Availability,
trim(upper(OwnClub_Old.Advance)) as Advance,
trim(upper(OwnClub_Old.'Dining Benefit')) as DiningBenefit,
trim(upper(OwnClub_Old.'Dining Availability')) as DiningAvailability
from OwnClub_Old)
select op.Community,op.Club,op.BenefitTier,
GolfBenefit ,
op.Availability ,
op.Advance ,
DiningBenefit ,
DiningAvailability
from op left join OwnClub_Old as old
on (op.Community=trim(upper(old.Community))
and op.Club=trim(upper(old.Club))
and op.BenefitTier=trim(upper(old.'Benefit Tier')))
where old.Community is null and old.Club is null and old.'Benefit Tier' is null"""

rs_benefit = pd.read_sql_query(query, conn)
rs_benefit.to_csv("outputs/diff_new.csv",index=False)

In [ ]:
query = """select *
         from OwnClub_Old inner join OwnClub_New on
         trim(upper(OwnClub_Old.Community)) = trim(upper(OwnClub_New.Community)) and 
         trim(upper(OwnClub_Old.Club)) = trim(upper(OwnClub_New.Club)) and
         trim(upper(OwnClub_Old.'Benefit Tier')) = trim(upper(OwnClub_New.'Benefit Tier'))
         where trim(upper(OwnClub_Old.'Golf Benefit')) = trim(upper(OwnClub_New.'Golf Benefit')) and 
         trim(upper(OwnClub_Old.Availability)) = trim(upper(OwnClub_New.Availability)) and
         trim(upper(OwnClub_Old.Advance)) = trim(upper(OwnClub_New.Advance)) and 
         trim(upper(OwnClub_Old.'Dining Benefit')) = trim(upper(OwnClub_New.'Dining Benefit')) and
         trim(upper(OwnClub_Old.'Dining Availability')) = trim(upper(OwnClub_New.'Dining Availability'))"""

rs_benefit = pd.read_sql_query(query, conn)
rs_benefit.to_csv("outputs/commonLines.csv",index=False)

In [ ]:
query = """select 
		 trim(upper(OwnClub_New.Community)) ,  
		 trim(upper(OwnClub_New.Club)) , 
         trim(upper(OwnClub_New.'Benefit Tier')) ,
		 trim(upper(OwnClub_New.'Golf Benefit')),
		 trim(upper(OwnClub_New.Availability)),
		 trim(upper(OwnClub_New.Advance)),
		 trim(upper(OwnClub_New.'Dining Benefit')),
		 trim(upper(OwnClub_New.'Dining Availability'))
		 from OwnClub_New
		 
		 except
		 
		 select 
		 trim(upper(OwnClub_Old.Community)) ,  
		 trim(upper(OwnClub_Old.Club)) , 
         trim(upper(OwnClub_Old.'Benefit Tier')) ,
		 trim(upper(OwnClub_Old.'Golf Benefit')),
		 trim(upper(OwnClub_Old.Availability)),
		 trim(upper(OwnClub_Old.Advance)),
		 trim(upper(OwnClub_Old.'Dining Benefit')),
		 trim(upper(OwnClub_Old.'Dining Availability'))
		 from OwnClub_Old
		 """

rs_benefit = pd.read_sql_query(query, conn)
rs_benefit.to_csv("outputs/diff.csv",index=False)

In [ ]:
query = """select 
		 trim(upper(OwnClub_Old.Community)) ,  
		 trim(upper(OwnClub_Old.Club)) , 
         trim(upper(OwnClub_Old.'Benefit Tier')) ,
		 trim(upper(OwnClub_Old.'Golf Benefit')),
		 trim(upper(OwnClub_Old.Availability)),
		 trim(upper(OwnClub_Old.Advance)),
		 trim(upper(OwnClub_Old.'Dining Benefit')),
		 trim(upper(OwnClub_Old.'Dining Availability'))
		 from OwnClub_Old
		 
		 except
		 
		 select 
		 trim(upper(OwnClub_New.Community)) ,  
		 trim(upper(OwnClub_New.Club)) , 
         trim(upper(OwnClub_New.'Benefit Tier')) ,
		 trim(upper(OwnClub_New.'Golf Benefit')),
		 trim(upper(OwnClub_New.Availability)),
		 trim(upper(OwnClub_New.Advance)),
		 trim(upper(OwnClub_New.'Dining Benefit')),
		 trim(upper(OwnClub_New.'Dining Availability'))
		 from OwnClub_New"""

rs_benefit = pd.read_sql_query(query, conn)
rs_benefit.to_csv("outputs/diff1.csv",index=False)

In [ ]:
query = """select *
         from OwnClub_Old inner join OwnClub_New on
         trim(upper(OwnClub_Old.Community)) = trim(upper(OwnClub_New.Community)) and 
         trim(upper(OwnClub_Old.Club)) = trim(upper(OwnClub_New.Club)) and
         trim(upper(OwnClub_Old.'Benefit Tier')) = trim(upper(OwnClub_New.'Benefit Tier'))
         where trim(upper(OwnClub_Old.'Golf Benefit')) = trim(upper(OwnClub_New.'Golf Benefit')) or 
         trim(upper(OwnClub_Old.Availability)) != trim(upper(OwnClub_New.Availability)) or
         trim(upper(OwnClub_Old.Advance)) != trim(upper(OwnClub_New.Advance)) or 
         trim(upper(OwnClub_Old.'Dining Benefit')) != trim(upper(OwnClub_New.'Dining Benefit')) or
         trim(upper(OwnClub_Old.'Dining Availability')) != trim(upper(OwnClub_New.'Dining Availability'))"""

rs_benefit = pd.read_sql_query(query, conn)
rs_benefit.to_csv("outputs/modifiedLines.csv",index=False)

In [ ]:
#benefit_dining_df_old = df_old[['Community','Benefit Tier','Dining Benefit']].drop_duplicates()
benefit_dining_df_old = df_old[['Community','Benefit Tier','Dining Benefit']].drop_duplicates()
benefit_dining_df_old = benefit_dining_df_old[(~benefit_dining_df_old['Dining Benefit'].isin(['No access','Home Club','NO ACCESS','NAN','N/A']))]
dining_benefit_old = list(benefit_dining_df_old.itertuples(index=False))

#benefit_dining_df_new = df_new[['Community','Benefit Tier','Dining Benefit']].drop_duplicates()
benefit_dining_df_new = df_new[['Community','Benefit Tier','Dining Benefit']].drop_duplicates()
benefit_dining_df_new = benefit_dining_df_new[(~benefit_dining_df_new['Dining Benefit'].isin(['No access','Home Club','NO ACCESS','NAN','N/A']))]
dining_benefit_new = list(benefit_dining_df_new.itertuples(index=False))

added_Ben_Dining = set(dining_benefit_new) -  set(dining_benefit_old)
#print(added_Ben_Golf)
print("Dining Benefit Added : " + str(len(added_Ben_Dining)))

dropped_Ben_Dining = set(dining_benefit_old) -  set(dining_benefit_new)
print("Dining Benefit Deprected : " + str(len(dropped_Ben_Dining)))

In [ ]:
accessrule_golf_df_old = df_old[['Community','Benefit Tier','Golf Benefit','Advance','Availability']].drop_duplicates()
#accessrule_golf_df_old = df_old[['Village','Benefit Tier','Golf Benefit','Advance','Availability']].drop_duplicates()

accessrule_golf_df_old = accessrule_golf_df_old[(~accessrule_golf_df_old['Golf Benefit'].isin(['No access','Home Club','NO ACCESS','NAN','N/A']))]
golf_ar_old = list(accessrule_golf_df_old.itertuples(index=False))

accessrule_golf_df_new = df_new[['Community','Benefit Tier','Golf Benefit','Advance','Availability']].drop_duplicates()
#accessrule_golf_df_new = df_new[['Village','Benefit Tier','Golf Benefit','Advance','Availability']].drop_duplicates()
accessrule_golf_df_new = accessrule_golf_df_new[(~accessrule_golf_df_new['Golf Benefit'].isin(['No access','Home Club','NO ACCESS','NAN','N/A']))]
golf_ar_new = list(accessrule_golf_df_new.itertuples(index=False))

added_ar_Golf = set(golf_ar_new) -  set(golf_ar_old)
#print(added_Ben_Golf)
print("Golf Access Rule Created : " + str(len(added_ar_Golf)))

dropped_ar_Golf = set(golf_ar_old) -  set(golf_ar_new)
print("Golf Access Rule Deprected : " + str(len(dropped_ar_Golf)))

In [ ]:
ar_dining_df_old = df_old[['Community','Benefit Tier','Dining Benefit','Dining Availability']].drop_duplicates()
#ar_dining_df_old = df_old[['Village','Benefit Tier','Dining Benefit','Dining Availability']].drop_duplicates()
ar_dining_df_old = ar_dining_df_old[(~ar_dining_df_old['Dining Benefit'].isin(['No access','Home Club','NO ACCESS','NAN','N/A']))]
dining_ar_old = list(ar_dining_df_old.itertuples(index=False))

ar_dining_df_new = df_new[['Community','Benefit Tier','Dining Benefit','Dining Availability']].drop_duplicates()
#ar_dining_df_new = df_new[['Village','Benefit Tier','Dining Benefit','Dining Availability']].drop_duplicates()
ar_dining_df_new = ar_dining_df_new[(~ar_dining_df_new['Dining Benefit'].isin(['No access','Home Club','NO ACCESS','NAN','N/A']))]
dining_ar_new = list(ar_dining_df_new.itertuples(index=False))

added_ar_Dining = set(dining_ar_new) -  set(dining_ar_old)
#print(added_Ben_Golf)
print("Dining Access Rule Created : " + str(len(added_ar_Dining)))

dropped_ar_Dining = set(dining_ar_old) -  set(dining_ar_new)
print("Dining Access Rule Deprected : " + str(len(dropped_ar_Dining)))

In [ ]:
count_data = [['Benefit Package Added', str(len(added_BP))],['Benefit Package Deprected', str(len(dropped_BP))], ['Golf Benefit Added', str(len(added_Ben_Golf))], ['Golf Benefit Deprecated', str(len(dropped_Ben_Golf))], ['Dining Benefit Added', str(len(added_Ben_Dining))], ['Dining Benefit Deprecated', str(len(dropped_Ben_Dining))], ['Golf Access Rule Added', str(len(added_ar_Golf))], ['Golf Access Rule Deprecated', str(len(dropped_ar_Golf))], ['Dining Access Rule Added', str(len(added_ar_Dining))], ['Dining Access Rule Deprecated', str(len(dropped_ar_Dining))]]
count_data_df = pd.DataFrame(count_data, columns = ['Description', 'Count'])
count_data_df.to_csv("outputs/ExpectedCountData.csv",index=False)                                                                       